In [3]:
import pandas as pd
import numpy as np
import json
import re

df_2018 = pd.read_csv('itpas2_2018_contributions2candidates.csv')

C:\Users\lucky\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10,11,12,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_2018.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,Unnamed: 22
0,C00177436,N,M2,P,2.020000e+17,24K,CCM,"YODER FOR CONGRESS, INC",OVERLAND PARK,KS,...,1172017.0,1500,C00472365,H0KS03137,40458934,1147350,NaN,NaN,4.02e+18,NaN
1,C00242271,N,M2,P,2.020000e+17,24K,CCM,KEVIN MCCARTHY FOR CONGRESS,BAKERSFIELD,CA,...,1182017.0,5000,C00420935,H6CA22125,74716003,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
2,C00242271,N,M2,P,2.020000e+17,24K,CCM,TIBERI FOR CONGRESS,COLUMBUS,OH,...,1182017.0,5000,C00347492,H0OH12062,74716004,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
3,C00242271,N,M2,P,2.020000e+17,24K,CCM,JEFF FLAKE FOR US SENATE INC,TEMPE,AZ,...,1182017.0,2500,C00347260,S2AZ00141,74716067,1147780,NaN,CONTRIBUTION,4.02e+18,NaN
4,C00051979,N,M2,P,2.020000e+17,24K,CCM,KAINE FOR VIRGINIA,ALEXANDRIA,VA,...,1232017.0,1000,C00495358,S2VA00142,8481131,1147789,NaN,CONTRIBUTION,4.02e+18,NaN


In [12]:
del pcap_data

In [5]:
src_dst = df_2018[['CMTE_ID', 'CAND_ID', 'STATE','TRANSACTION_AMT']]

In [6]:
src_dst.head()

,CMTE_ID,CAND_ID,STATE,TRANSACTION_AMT
0,C00177436,H0KS03137,KS,1500
1,C00242271,H6CA22125,CA,5000
2,C00242271,H0OH12062,OH,5000
3,C00242271,S2AZ00141,AZ,2500
4,C00051979,S2VA00142,VA,1000


In [7]:
src_dst.shape

(237090, 4)

Aggregate Records

In [83]:
grouped_src_dst = src_dst.groupby(['CMTE_ID', 'CAND_ID','STATE'])['TRANSACTION_AMT'].sum().reset_index()
#grouped_src_dst['TRANSACTION_AMT'].agg(np.sum)
grouped_src_dst.head()

,CMTE_ID,CAND_ID,STATE,TRANSACTION_AMT
0,C00000059,H0KS03137,KS,5000
1,C00000059,H0MO06073,MO,6000
2,C00000059,H2MA09072,MA,2500
3,C00000059,H2NC11080,NC,6000
4,C00000059,H2PA17079,PA,2000


In [84]:
grouped_src_dst.shape

(125932, 4)

In [89]:
s = grouped_src_dst['TRANSACTION_AMT']
s.isnull().values.any()

False

Format records

In [90]:
links_list = list(grouped_src_dst.apply(lambda row: {"source": row['CMTE_ID'], "target": row['CAND_ID'], "value": row['TRANSACTION_AMT']}, axis=1))

In [91]:
links_list[1:1000]

[{'source': 'C00000059', 'target': 'H0MO06073', 'value': 6000},
 {'source': 'C00000059', 'target': 'H2MA09072', 'value': 2500},
 {'source': 'C00000059', 'target': 'H2NC11080', 'value': 6000},
 {'source': 'C00000059', 'target': 'H2PA17079', 'value': 2000},
 {'source': 'C00000059', 'target': 'H4MO05234', 'value': 7000},
 {'source': 'C00000059', 'target': 'H4MO08162', 'value': 3000},
 {'source': 'C00000059', 'target': 'H6FL13148', 'value': 1000},
 {'source': 'C00000059', 'target': 'H6KS01179', 'value': 1000},
 {'source': 'C00000059', 'target': 'H6MD07160', 'value': 5000}]

In [58]:
#grouped_src_dst2 = src_dst.groupby(['CMTE_ID', 'STATE'])
#grouped_src_dst2.head()
df_nodes_C = pd.DataFrame(src_dst.CMTE_ID.unique())
df_nodes_C["group"] = "Committee"
df_nodes_C.head()

,0,group
0,C00177436,Committee
1,C00242271,Committee
2,C00051979,Committee
3,C00478099,Committee
4,C00280222,Committee


In [59]:
df_nodes_P = pd.DataFrame(src_dst.CAND_ID.unique())
df_nodes_P["group"] = "Politician"
df_nodes_P.head()

,0,group
0,H0KS03137,Politician
1,H6CA22125,Politician
2,H0OH12062,Politician
3,S2AZ00141,Politician
4,S2VA00142,Politician


In [99]:
df_nodes = pd.concat([df_nodes_P,df_nodes_C])
df_nodes.head()

,0,group
0,H0KS03137,Politician
1,H6CA22125,Politician
2,H0OH12062,Politician
3,S2AZ00141,Politician
4,S2VA00142,Politician


In [100]:
len(df_nodes)

6083

In [101]:
df_nodes.dropna()

,0,group
0,H0KS03137,Politician
1,H6CA22125,Politician
2,H0OH12062,Politician
3,S2AZ00141,Politician
4,S2VA00142,Politician
5,H0VA02118,Politician
6,S0WI00197,Politician
7,S6VA00093,Politician
8,S6UT00063,Politician
9,H0IN02125,Politician


In [104]:
s = df_nodes['group']
s.isnull().values.any()

False

In [106]:
node_list = list(df_nodes.apply(lambda row: {"name": row[0], "group": row['group']}, axis=1))

In [107]:
len(node_list)

6083

In [108]:
#node_list = list(grouped_src_dst.apply(lambda row: {"name": row['CAND_ID'], "name": row['CMTE_ID'],"group": row['STATE']}, axis=1))

In [69]:
node_list[-5:]

[{'group': 'Committee', 'name': 'C00678920'},
 {'group': 'Committee', 'name': 'C90017633'},
 {'group': 'Committee', 'name': 'C90017690'},
 {'group': 'Committee', 'name': 'C90017799'},
 {'group': 'Committee', 'name': 'C00130658'}]

In [67]:
node_list

[{'group': 'Politician', 'name': 'H0KS03137'},
 {'group': 'Politician', 'name': 'H6CA22125'},
 {'group': 'Politician', 'name': 'H0OH12062'},
 {'group': 'Politician', 'name': 'S2AZ00141'},
 {'group': 'Politician', 'name': 'S2VA00142'},
 {'group': 'Politician', 'name': 'H0VA02118'},
 {'group': 'Politician', 'name': 'S0WI00197'},
 {'group': 'Politician', 'name': 'S6VA00093'},
 {'group': 'Politician', 'name': 'S6UT00063'},
 {'group': 'Politician', 'name': 'H0IN02125'},
 {'group': 'Politician', 'name': 'S0FL00338'},
 {'group': 'Politician', 'name': 'H2AZ08102'},
 {'group': 'Politician', 'name': 'H4FL26038'},
 {'group': 'Politician', 'name': 'H2IN05082'},
 {'group': 'Politician', 'name': 'H0WA03187'},
 {'group': 'Politician', 'name': 'H0IL11052'},
 {'group': 'Politician', 'name': 'H0AL02087'},
 {'group': 'Politician', 'name': 'H0PA06076'},
 {'group': 'Politician', 'name': 'H0NY29054'},
 {'group': 'Politician', 'name': 'H8FL16022'},
 {'group': 'Politician', 'name': 'H0PA03271'},
 {'group': 'P

In [109]:
json_prep = {"nodes":node_list, "links":links_list}

json_prep.keys()
   # dict_keys(['links', 'nodes'])

dict_keys(['nodes', 'links'])

In [110]:
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [111]:
filename_out = 'test_export.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()